### Dataset 

In [ ]:
import os
import numpy as np
import cv2
import mediapipe as mp
import time
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

In [25]:
import sys  
sys.path.insert(0, 'human_activity_recognition--main/new')
from pose_media import mediapipe_pose

In [26]:
DATA_PATH = "./DATA/" 
if not os.path.exists(DATA_PATH):
    os.mkdir(DATA_PATH)   

In [27]:
mp = mediapipe_pose()

In [28]:
cTime,pTime = 0,0

In [29]:
sequence_length = 30
path = 'C:/Users/sanka/OneDrive/Desktop/Minor_project/Dataset/depth/'
actions = os.listdir(path)
for action in actions:
    if not os.path.exists(DATA_PATH+action):
        os.mkdir(DATA_PATH+action)
    video_list = os.listdir(path+action)
    no_sequences = 1776//len(video_list)
    extra_sequences = 1776%len(video_list)
    for video in range(len(video_list)):
        cd = os.path.join(path+ action+"/" + video_list[video])
        cap = cv2.VideoCapture(cd)
        with mp.mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
            if video < extra_sequences:
                no_sequences += 1
                pre_sequences = video * no_sequences 
            elif video == extra_sequences:
                pre_sequences = video * (no_sequences +1)
            else:
                pre_sequences = extra_sequences * (no_sequences +1) + (video - extra_sequences)*no_sequences
            for sequence in range(no_sequences):
                if not os.path.exists(DATA_PATH+action+"/"+str(pre_sequences+sequence)):
                    os.mkdir(DATA_PATH+action+"/"+str(pre_sequences+sequence))
                for frame_num in range(sequence_length):
                        ref,frame = cap.read()
                        try:
                            image, results = mp.mediapipe_detection(frame, holistic)
                        except:
                            break
                            
                        mp.draw_styled_landmarks(image, results)
                        cTime = time.time()
                        fps = 1 / (cTime - pTime)
                        pTime = cTime
                        cv2.putText(image,"FPS:" +str(int(fps)),(10,100), cv2.FONT_HERSHEY_PLAIN, 2,(255,0,190),2,cv2.LINE_AA)

                        if frame_num == 0: 
                            cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, video), (5,15), 
                                              cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 0, 255), 1, cv2.LINE_AA)
                            cv2.imshow('OpenCV Feed', image)
                        else: 
                            cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action,video), (5,15), 
                                            cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 0, 255), 1, cv2.LINE_AA)
                            cv2.imshow('OpenCV Feed', image)

                        keypoints = mp.extract_keypoints(results)
                        npy_path = os.path.join(DATA_PATH, action, str(pre_sequences+sequence), str(frame_num))
                        np.save(npy_path, keypoints)
                        if cv2.waitKey(1) & 0xFF == ord("q"):
                            break
cap.release()
cv2.destroyAllWindows()

In [85]:
DATA_PATH = os.path.join('human_activity_recognition--main\\DATA') 
actions = np.array(["Salute","Sitting"])
no_sequences = 1776
sequence_length = 30

In [86]:
label_map = {label:num for num,label in enumerate(actions)}
label_map

{'Salute': 0, 'Sitting': 1}

In [87]:
def load_data(actions,no_sequences,sequence_length):
    sequences1,labels = [],[]
    for action in actions:
        for sequence in range(no_sequences):
            window = []
            for frame_num in range(sequence_length):
                res = np.load(os.path.join(DATA_PATH,action,str(sequence),"{}.npy".format(frame_num)))
                window.append(res)
            sequences1.append(window)
            labels.append(label_map[action])
    return sequences1,labels

In [88]:
# sequences,labels = [],[]
# for action in os.listdir("xxx/"):
#     fd = os.path.join("xxx/" + action)
#     window = []
#     for folder in os.listdir(fd):
#         fd1 = os.path.join("xxx/" + action +"/"+ folder)
#         for file in os.listdir(fd1):
#             res = np.load(os.path.join("xxx/" + action +"/"+ folder + "/" + file))
#             window.append(res)
#         sequences.append(window)
#         labels.append(label_map[action])

In [89]:
%time sequences,labels = load_data(actions,no_sequences,sequence_length)

FileNotFoundError: [Errno 2] No such file or directory: 'human_activity_recognition--main\\DATA\\Salute\\0\\0.npy'

In [90]:
X = np.array(sequences)
X.shape

NameError: name 'sequences' is not defined

In [91]:
y = to_categorical(labels).astype(int)
y

NameError: name 'labels' is not defined

In [92]:
with open('X1.npy', 'wb') as f:
    np.save(f, X)
with open('y1.npy', 'wb') as f:
    np.save(f, y)

NameError: name 'X' is not defined